# Training GNN and Inference
This notebook outlines the steps train a heterogeneous GNN model with city graph objects for urban scale building operating energy prediction. The accompanying data files can be downloaded on Figshare: https://doi.org/10.6084/m9.figshare.28188242.v1. Our example covers Seattle.

We first preprocess our data through standard normalization and scaling. Next, we load the multi-modal model architecture and instantiate data loaders. Finally, we train our GNN model and store the weights. For inference, we provide a pretrained weight file which can be used directly for inference. 

### Import Libraries

In [1]:
import glob
import torch
from tqdm import tqdm
from torch import nn
import random
import pandas as pd
import geopandas as gpd
import numpy as np
import torch_geometric.transforms as T
import torch.nn.functional as F
import matplotlib.pyplot as plt
from torch_geometric.data import HeteroData
from torch_geometric.loader import NeighborLoader, HGTLoader
from sklearn.preprocessing import StandardScaler
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch_geometric.nn import HeteroConv, GCNConv, SAGEConv, GATv2Conv, Linear, GATConv

### Load data

In [2]:
# Load node data
building_nodes = pd.read_parquet('../seattle_files/Seattle_Graphs/seattle_buildings.parquet')
building_nodes = building_nodes.reset_index()
intersection_nodes = pd.read_parquet('../seattle_files/Seattle_Graphs/seattle_intersections.parquet')
street_nodes = pd.read_parquet('../seattle_files/Seattle_Graphs/seattle_streets.parquet')
plot_nodes = pd.read_parquet('../seattle_files/Seattle_Graphs/seattle_plots.parquet')

In [3]:
# Load edge data
building2building = np.loadtxt('../seattle_files/Seattle_Graphs/building_to_building_edges.txt').astype(int)
plot2building = np.loadtxt('../seattle_files/Seattle_Graphs/building_to_plot_edges.txt').astype(int)
intersection2street = np.loadtxt('../seattle_files/Seattle_Graphs/intersection_to_street_edges.txt').astype(int)
plot2plot = np.loadtxt('../seattle_files/Seattle_Graphs/plot_to_plot_edges.txt').astype(int)
street2plot = np.loadtxt('../seattle_files/Seattle_Graphs/plot_to_street_edges.txt').astype(int)[[1, 0]]
street2building = np.loadtxt('../seattle_files/Seattle_Graphs/street_to_building_edges.txt').astype(int)

# Remove intersecting building edges
building2building = building2building[:,np.where(building2building[1] <= building2building[0].max())]
building2building = building2building.squeeze(1)
plot2building = plot2building[:,np.where(plot2building[1] <= building2building[0].max())]
plot2building = plot2building.squeeze(1)
street2building = street2building[:,np.where(street2building[1] <= building2building[0].max())]
street2building = street2building.squeeze(1)

In [4]:
# Load energy data
baseline_df = pd.read_csv('../energy_data/building_energy_labels.csv')
baseline_df['bid'] = baseline_df['bid'].astype(str)

In [5]:
train_bids = np.loadtxt('../energy_data/train_bids', dtype = str)
train_set = baseline_df[baseline_df['bid'].isin(train_bids)]
val_bids = np.loadtxt('../energy_data/val_bids', dtype = str)
val_set = baseline_df[baseline_df['bid'].isin(val_bids)]
test_bids = np.loadtxt('../energy_data/test_bids', dtype = str)
test_set = baseline_df[baseline_df['bid'].isin(test_bids)]

In [ ]:
print(len(train_set))
print(len(val_set))
print(len(test_set))

In [7]:
def fill_na_in_df(df):
    na_cols = []
    for col in df.columns:
        if sum(df[col].isna()) != 0:
            na_cols.append(col)
    
    for missing_col in na_cols:
        temp_mean = df[missing_col].mean()
        df[missing_col].fillna(value=temp_mean, inplace=True)
        
    return df

In [8]:
# Fill na with mean value
building_nodes = fill_na_in_df(building_nodes)
intersection_nodes = fill_na_in_df(intersection_nodes)
street_nodes = fill_na_in_df(street_nodes)
plot_nodes = fill_na_in_df(plot_nodes)

In [9]:
baseline_df = fill_na_in_df(baseline_df)

### Prepare target labels

In [10]:
# Prepare target tensor
mapping_dict = {}
for i, bid in enumerate(building_nodes['bid'].values):
    mapping_dict[bid] = i
    
y_target = np.zeros(len(building_nodes))

# Prepare sequence of indexes
sequences = []
for bid in baseline_df['bid'].values:
    sequences.append(mapping_dict[str(bid)])
    
for i, idx in enumerate(sequences):
    y_target[idx] = baseline_df.iloc[[i]]['Total Carbon'].values[0]

y_target_log = np.log(y_target+1)
    
# Set random seed
torch.manual_seed(0)
random.seed(0)
np.random.seed(0)

# Instantiate empty vector masks
train_mask = np.zeros(len(building_nodes)).astype(int)
val_mask = np.zeros(len(building_nodes)).astype(int)
test_mask = np.zeros(len(building_nodes)).astype(int)

train_idx = np.loadtxt('../energy_data/train_idx', dtype=int)
train_idx = list(train_idx)
val_idx = np.loadtxt('../energy_data/val_idx', dtype=int)
val_idx = list(val_idx)
test_idx = np.loadtxt('../energy_data/test_idx', dtype=int)
test_idx = list(test_idx) 

# Assign 1 to entries corresponding to each index
np.put(train_mask, train_idx, 1)
np.put(val_mask, val_idx, 1)
np.put(test_mask, test_idx, 1)

# # Split entries into train, val, test
# # train_idx, val_idx, test_idx = sequences[:int(0.7*len(sequences))], sequences[int(0.7*len(sequences)):int(0.85*len(sequences))], sequences[int(0.85*len(sequences)):]

### Check train, val, and test indices

In [11]:
assert(sum(train_mask[train_idx]) == len(train_mask[train_idx]))
assert(sum(val_mask[val_idx]) == len(val_mask[val_idx]))
assert(sum(test_mask[test_idx]) == len(test_mask[test_idx]))
assert(building_nodes.isnull().values.any() == False)
assert(intersection_nodes.isnull().values.any() == False)
assert(street_nodes.isnull().values.any() == False)
assert(plot_nodes.isnull().values.any() == False)
assert(baseline_df.isnull().values.any() == False)

### Remove non-numerical columns and standardise

In [12]:
building_nodes_copy = building_nodes.copy()

In [13]:
building_nodes_copy = building_nodes_copy.set_index('bid')

In [14]:
intersection_nodes = intersection_nodes.drop(['intersection_id', 'osmid', 'x', 'y', 'geometry'], axis=1)

In [15]:
street_nodes = street_nodes.drop(['edge_id', 'geometry'], axis=1)

In [16]:
plot_nodes = plot_nodes.drop(['plot_id', 'geometry'], axis=1)

In [17]:
# Get one hot encoding of columns B
plot_nodes_num = pd.get_dummies(plot_nodes['plot_lcz'], prefix='lcz')
plot_nodes = plot_nodes.drop(['plot_lcz'], axis=1)
plot_nodes = plot_nodes.join(plot_nodes_num)

### Standardise columns

In [18]:
from sklearn.compose import ColumnTransformer

In [19]:
scale = StandardScaler()

In [20]:
ct = ColumnTransformer([
        ('somename', StandardScaler(), ['plot_area', 'plot_perimeter', 'plot_building_count',
       'plot_building_area_mean', 'plot_building_area_std',
       'plot_building_perimeter_mean', 'plot_building_perimeter_std',
       'plot_building_circ_compact_mean', 'plot_building_circ_compact_std',
       'plot_building_convexity_mean', 'plot_building_convexity_std',
       'plot_building_corners_mean', 'plot_building_corners_std',
       'plot_building_elongation_mean', 'plot_building_elongation_std',
       'plot_building_orientation_mean', 'plot_building_orientation_std',
       'plot_building_longest_axis_length_mean',
       'plot_building_longest_axis_length_std', 'plot_building_eri_mean',
       'plot_building_eri_std', 'plot_building_fractaldim_mean',
       'plot_building_fractaldim_std', 'plot_building_rectangularity_mean',
       'plot_building_rectangularity_std', 'plot_building_squareness_mean',
       'plot_building_squareness_std', 'plot_building_square_compactness_mean',
       'plot_building_square_compactness_std', 'plot_building_shape_idx_mean',
       'plot_building_shape_idx_std', 'plot_building_complexity_mean',
       'plot_building_complexity_std', 'plot_building_total_area',
       'plot_building_built_coverage', 'plot_circ_compact', 'plot_convexity',
       'plot_corners', 'plot_elongation', 'plot_orientation',
       'plot_longest_axis_length', 'plot_eri', 'plot_fractaldim',
       'plot_rectangularity', 'plot_square_compactness', 'plot_shape_idx',
       'plot_squareness', 'plot_complexity', 'Civic', 'Commercial',
       'Entertainment', 'Food', 'Healthcare', 'Institutional', 'Recreational',
       'Social', 'PopSum', 'Men', 'Women', 'Elderly', 'Youth', 'Children',
       'subzone_mean_Green_View', 'subzone_std_Green_View',
       'subzone_mean_Sky_View', 'subzone_std_Sky_View',
       'subzone_mean_Building_View', 'subzone_std_Building_View',
       'subzone_mean_Road_View', 'subzone_std_Road_View',
       'subzone_mean_Visual_Complexity', 'subzone_std_Visual_Complexity'])
    ], remainder='passthrough')

In [21]:
scaled_building_nodes = scale.fit_transform(building_nodes_copy)
scaled_intersection_nodes = scale.fit_transform(intersection_nodes)
scaled_plot_nodes = ct.fit_transform(plot_nodes)
scaled_street_nodes = scale.fit_transform(street_nodes)

### Add Image Pipeline

In [22]:
import os
import torchvision
from PIL import Image
from torchvision.transforms import v2
from torchvision import transforms, utils
from torch.utils.data import Dataset, DataLoader

### Define Model

In [ ]:
class HeteroSAGE(torch.nn.Module):
    def __init__(self, hidden_channels, out_channels):
        super().__init__()

        self.conv1 = HeteroConv({
                ('intersection', 'to', 'street'): SAGEConv((-1, -1), hidden_channels),
                ('urban_plot','to','building'): SAGEConv((-1, -1), hidden_channels),
                ('urban_plot', 'to', 'urban_plot'): SAGEConv((-1, -1), hidden_channels),
                ('building', 'to', 'building'): SAGEConv((-1, -1), hidden_channels),
                ('street', 'to', 'urban_plot'): SAGEConv((-1, -1), hidden_channels), 
                ('street', 'to', 'building'): SAGEConv((-1, -1), hidden_channels),
            }, aggr='sum')
        
        self.conv2 = HeteroConv({
                ('urban_plot','to','building'): SAGEConv((-1, -1), hidden_channels),
                ('building', 'to', 'building'): SAGEConv((-1, -1), hidden_channels),
                ('street', 'to', 'building'): SAGEConv((-1, -1), hidden_channels),
            }, aggr='sum')
        
        self.fc1 = Linear(hidden_channels, out_channels)
        # self.bn1 = BatchNorm(hidden_channels)
        # self.bn2 = BatchNorm(hidden_channels)

    def forward(self, x_dict, edge_index_dict):
        x_dict = self.conv1(x_dict, edge_index_dict)
        # x_dict = {key: self.bn1(x) for key, x in x_dict.items()}
        x_dict = {key: x.relu() for key, x in x_dict.items()}
        x_dict = {key: F.dropout(x, p=0.5, training=self.training) for key, x in x_dict.items()}
        x_dict = self.conv2(x_dict, edge_index_dict)
        # x_dict = {key: self.bn2(x) for key, x in x_dict.items()}
        x_dict = {key: x.relu() for key, x in x_dict.items()}
        x_dict = {key: F.dropout(x, p=0.5, training=self.training) for key, x in x_dict.items()}
        out = self.fc1(x_dict['building'])
        return out
        
class MultiModal(torch.nn.Module):
    def __init__(self, het_graph_model, image_model, combined_channel, out_channel):
        super().__init__()
        
        self.graph_model = het_graph_model
        self.image_model = image_model
        
        self.fc1 = Linear(combined_channel, int(combined_channel/2))
        self.fc2 = Linear(int(combined_channel/2), out_channel)
    
    def forward(self, x_dict, edge_index_dict, image_tensor):
        out1 = self.graph_model(x_dict, edge_index_dict)[:batch_size]
        try:
            out2 = self.image_model(image_tensor).logits
        except AttributeError: 
            out2 = self.image_model(image_tensor)
        combined = torch.concatenate((out1, out2), 1)
        combined = self.fc1(combined)
        final = self.fc2(combined)
        return final

H, W = 512, 512
transforms = v2.Compose([
    v2.Resize([H,W]),
    v2.ToTensor()
])

class ImageDataset(Dataset):
    """Custom image dataset."""

    def __init__(self, root_dir, choice=None, transform=None):
        """
        Arguments:
            root_dir (string): Directory with all the images.
            choice (list): Accepts 'train', 'val', or 'test'
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.root_dir = root_dir
        self.transform = transform
        self.choice = choice
        self.targets = None
        
        if choice in ['train', 'test', 'val']:
            bids = np.loadtxt(f'../energy_data/{self.choice}_bids', dtype='str')
            self.paths = [os.path.join(self.root_dir, f'{bid}') for bid in bids]
            # self.targets = np.loadtxt(os.path.join(os.getcwd(), 'data', f'{self.choice}_targets'))
        if choice == 'pseudo_train':
            bids = np.loadtxt(f'../energy_data/{self.choice}_bids', dtype='str')
            self.paths = [os.path.join(self.root_dir, f'{bid}') for bid in bids]
        if choice == 'pseudo_train_val':
            bids = np.loadtxt(f'../energy_data/{self.choice}_bids', dtype='str')
            self.paths = [os.path.join(self.root_dir, f'{bid}') for bid in bids]
        if choice is None:
            bids =  all_bids
            self.paths = [os.path.join(self.root_dir, f'{bid}') for bid in bids]

    def __len__(self):
        return len(self.paths)
        
    def __getitem__(self, idx):
        if isinstance(idx,int):
            img_path = self.paths[idx] + '.png'
            image = Image.open(img_path)
            # target = self.targets[idx]
            if self.transform:
                image = self.transform(image)
                image = torch.unsqueeze(image, 0)
        
            return image
        
        if len(idx) > 1:
            temp = torch.Tensor()
            # target_list = []
            for ind in idx:
                img_path = self.paths[ind] + '.png'
                image = Image.open(img_path)
                # target = self.targets[ind]
                # target_list.append(target)
                if self.transform:
                    image = self.transform(image)
                    image = torch.unsqueeze(image, 0)
                temp = torch.concat((temp, image), dim=0)
                
            return temp
                
train_image_data = ImageDataset('../seattle_files//seattle_buildings/', choice = 'train', transform = transforms)
val_image_data = ImageDataset('./seattle_buildings/', choice = 'val', transform = transforms)

In [ ]:

torch.manual_seed(0)
random.seed(0)
np.random.seed(0)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
hidden_dim = 32
lr = 0.00005
batch_size = 32
EPOCH = 25
image_features = 64
graph_features = 128
conv_layer = 'sage'
image_layer = 'resnet18'

# Data build
data = HeteroData()

# Build heterogeneous graphs
data['building'].x = torch.from_numpy(scaled_building_nodes.astype(np.float32))
data['street'].x = torch.from_numpy(scaled_street_nodes.astype(np.float32))
data['urban_plot'].x = torch.from_numpy(scaled_plot_nodes.astype(np.float32))
data['intersection'].x = torch.from_numpy(scaled_intersection_nodes.astype(np.float32))

# Insert edges
data['building','building'].edge_index = torch.from_numpy(building2building.astype(np.int64))
data['urban_plot', 'building'].edge_index = torch.from_numpy(plot2building.astype(np.int64))
data['intersection','street'].edge_index = torch.from_numpy(intersection2street.astype(np.int64))
data['urban_plot','urban_plot'].edge_index = torch.from_numpy(plot2plot.astype(np.int64))
data['street','urban_plot'].edge_index = torch.from_numpy(street2plot.astype(np.int64))
data['street','building'].edge_index = torch.from_numpy(street2building.astype(np.int64))

# Insert y target
# y_target_mod = y_target.copy()
# np.put(y_target_mod, train_idx, 3)
# np.put(y_target_mod, val_idx, 4)
# np.put(y_target_mod, test_idx, 5)
data['building'].y = torch.from_numpy(y_target_log.astype(np.float32))

# Insert train, val, and test masks
data['building'].train_mask = torch.from_numpy(train_mask).bool()
data['building'].val_mask = torch.from_numpy(val_mask).bool()
data['building'].test_mask = torch.from_numpy(test_mask).bool()

# Add self loop and undirected edges
# data = T.ToUndirected()(data)
data = T.AddSelfLoops()(data)

train_loader = NeighborLoader(data, num_neighbors=[8,8], batch_size=batch_size, input_nodes = ('building', data['building'].train_mask), shuffle=True, drop_last=True)
val_loader = NeighborLoader(data, num_neighbors=[8,8], batch_size=batch_size, input_nodes = ('building', data['building'].val_mask), shuffle=True, drop_last=True)

# Load Image Model
best_model = torch.load(f'../seattle_files/Image_Model/{image_layer}.pth')
best_model.fc = Linear(512, image_features)
    
# Load GNN Model
graph_model = HeteroSAGE(hidden_channels=hidden_dim, out_channels=graph_features)
graph_model = graph_model.to(device)

model = MultiModal(het_graph_model = graph_model, image_model=best_model, combined_channel=image_features+graph_features, out_channel=1).to(device)

# Initialise optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
scheduler = ReduceLROnPlateau(optimizer, 'min', patience=10, verbose=True)

# Train and validate
### BATCH TRAINING
val_loss_target = 0.8

for epoch in range(EPOCH):

    # Initialise epoch loss
    epoch_train_loss = 0
    epoch_val_loss = 0
    # Train model
    model.train()

    # Load train data batches
    for k, train_batch in enumerate(tqdm(train_loader)):
        optimizer.zero_grad()

        # Get training image tensor
        train_image_pos = []
        for i in train_batch['building']['n_id'][0:train_batch['building']['batch_size']]:
            train_image_pos.append(train_idx.index(i))

        # Send training batch to cuda
        train_batch = train_batch.to(device)
        train_image_tensor = train_image_data[train_image_pos].to(device)

        # Predict 
        pred = model(train_batch.x_dict, train_batch.edge_index_dict, train_image_tensor)

        # Compute loss
        train_loss = F.mse_loss(pred[:batch_size].squeeze(), train_batch['building'].y[:batch_size].squeeze())
        train_loss.backward()
        train_loss = train_loss.to('cpu')

        # Sum loss
        epoch_train_loss += train_loss.item() * batch_size
        
        # Update weights
        optimizer.step()
    epoch_train_loss /= len(train_bids)
    print(f'Epoch: {epoch} | Average Train Loss: {round(epoch_train_loss, 5)}')

    # Evaluate model
    model.eval()

    # Validate model
    with torch.no_grad():
        for k, val_batch in enumerate(tqdm(val_loader)):
            
            # Get validation image tensor
            val_image_pos = []
            for i in val_batch['building']['n_id'][0:val_batch['building']['batch_size']]:
                val_image_pos.append(val_idx.index(i))
            val_batch = val_batch.to(device)
            val_image_tensor = val_image_data[val_image_pos].to(device)

            # Predict 
            val_pred = model(val_batch.x_dict, val_batch.edge_index_dict, val_image_tensor)

            # Compute loss
            val_loss = F.mse_loss(val_pred[:batch_size].squeeze(), val_batch['building'].y[:batch_size].squeeze()).to('cpu')

            # Aggregate validation loss
            epoch_val_loss += val_loss.item() * batch_size
            
    epoch_val_loss = epoch_val_loss / len(val_bids)
    print(f'Epoch: {epoch} | Average Val Loss: {round(epoch_val_loss, 5)}')
        
    scheduler.step(epoch_val_loss)

    # Save checkpoint if model performs well
    if val_loss_target > epoch_val_loss:
        torch.save({
                    'epoch': epoch,
                    'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'loss': epoch_val_loss,
                    }, f'./output/{image_layer}_{hidden_dim}_{batch_size}_{image_features}_{graph_features}_{conv_layer}_{round(epoch_val_loss, 5)}.pt')
        val_loss_target = epoch_val_loss
        print('-----------------Saved model checkpoint-------------------!')


### Predict Loss on Test Set

In [ ]:
H, W = 512, 512
transforms = v2.Compose([
    v2.Resize([H,W]),
    v2.ToTensor()
])

class ImageDataset(Dataset):
    """Custom image dataset."""

    def __init__(self, root_dir, choice=None, transform=None):
        """
        Arguments:
            root_dir (string): Directory with all the images.
            choice (list): Accepts 'train', 'val', or 'test'
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.root_dir = root_dir
        self.transform = transform
        self.choice = choice
        self.targets = None
        
        if choice in ['train', 'test', 'val']:
            bids = np.loadtxt(os.path.join(os.getcwd(), 'energy_data', f'{self.choice}_bids'), dtype='str')
            self.paths = [os.path.join(os.getcwd(), self.root_dir, f'{bid}') for bid in bids]
            # self.targets = np.loadtxt(os.path.join(os.getcwd(), 'data', f'{self.choice}_targets'))
        if choice == 'pseudo_train':
            bids = np.loadtxt(os.path.join(os.getcwd(), 'energy_data', f'{self.choice}_bids'), dtype='str')
            self.paths = [os.path.join(os.getcwd(), self.root_dir, f'{bid}') for bid in bids]
        if choice == 'pseudo_train_val':
            bids = np.loadtxt(os.path.join(os.getcwd(), 'energy_data', f'{self.choice}_bids'), dtype='str')
            self.paths = [os.path.join(os.getcwd(), self.root_dir, f'{bid}') for bid in bids]
        if choice is None:
            bids =  all_bids
            self.paths = [os.path.join(os.getcwd(), self.root_dir, f'{bid}') for bid in bids]

    def __len__(self):
        return len(self.paths)
        
    def __getitem__(self, idx):
        if isinstance(idx,int):
            img_path = self.paths[idx] + '.png'
            image = Image.open(img_path)
            # target = self.targets[idx]
            if self.transform:
                image = self.transform(image)
                image = torch.unsqueeze(image, 0)
        
            return image
        
        if len(idx) > 1:
            temp = torch.Tensor()
            # target_list = []
            for ind in idx:
                img_path = self.paths[ind] + '.png'
                image = Image.open(img_path)
                # target = self.targets[ind]
                # target_list.append(target)
                if self.transform:
                    image = self.transform(image)
                    image = torch.unsqueeze(image, 0)
                temp = torch.concat((temp, image), dim=0)
                
            return temp
                
test_image_data = ImageDataset('./seattle_buildings/', choice = 'test', transform = transforms)

### Load model and weights

In [ ]:
model_path = '../sea_hetero/output/resnet18_32_32_64_128_sage_0.63699.pt'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

hidden_dim = int(model_path.split('_')[2])
batch_size = int(model_path.split('_')[3])
image_features = int(model_path.split('_')[4])
graph_features = int(model_path.split('_')[5])

### Initialise data and seed
torch.manual_seed(0)
random.seed(0)
np.random.seed(0)

data = HeteroData()

# Build heterogeneous graphs
data['building'].x = torch.from_numpy(scaled_building_nodes.astype(np.float32))
data['street'].x = torch.from_numpy(scaled_street_nodes.astype(np.float32))
data['urban_plot'].x = torch.from_numpy(scaled_plot_nodes.astype(np.float32))
data['intersection'].x = torch.from_numpy(scaled_intersection_nodes.astype(np.float32))

# Insert edges
data['building','building'].edge_index = torch.from_numpy(building2building.astype(np.int64))
data['urban_plot', 'building'].edge_index = torch.from_numpy(plot2building.astype(np.int64))
data['intersection','street'].edge_index = torch.from_numpy(intersection2street.astype(np.int64))
data['urban_plot','urban_plot'].edge_index = torch.from_numpy(plot2plot.astype(np.int64))
data['street','urban_plot'].edge_index = torch.from_numpy(street2plot.astype(np.int64))
data['street','building'].edge_index = torch.from_numpy(street2building.astype(np.int64))

# Insert y target
data['building'].y = torch.from_numpy(y_target_log.astype(np.float32))

# Insert train, val, and test masks
data['building'].train_mask = torch.from_numpy(train_mask).bool()
data['building'].val_mask = torch.from_numpy(val_mask).bool()
data['building'].test_mask = torch.from_numpy(test_mask).bool()

# Add self loop and undirected edges
# data = T.ToUndirected()(data)
data = T.AddSelfLoops()(data)

test_loader = NeighborLoader(data, num_neighbors=[8,8], batch_size=batch_size, input_nodes = ('building', data['building'].test_mask), shuffle=True, drop_last=True)

### Load Model
image_encoder = model_path.split('_')[0][9:]
gnn_encoder = model_path.split('_')[-2]

image_model = torch.load('./models/resnet18.pth')
image_model.fc = Linear(512, image_features)

graph_model = HeteroSAGE(hidden_channels=hidden_dim, out_channels=graph_features)

print(f'Loaded model with image encoder: {image_encoder} and graph encoder: {gnn_encoder}')

with torch.no_grad():  # Initialize lazy modules.
    out = graph_model(data.x_dict, data.edge_index_dict)[:batch_size]

graph_model = graph_model.to(device)

model = MultiModal(het_graph_model = graph_model, image_model=image_model, combined_channel=image_features+graph_features, out_channel=1).to(device)
# Initialise optimizer

checkpoint = torch.load(model_path)
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

### Test model
total_test_loss = 0
with torch.no_grad():
    for k, test_batch in enumerate(tqdm(test_loader)):
        
        # Get validation image tensor
        test_image_pos = []
        for i in test_batch['building']['n_id'][0:test_batch['building']['batch_size']]:
            test_image_pos.append(test_idx.index(i))
        test_batch = test_batch.to(device)
        test_image_tensor = test_image_data[test_image_pos].to(device)

        # Predict 
        test_pred = model(test_batch.x_dict, test_batch.edge_index_dict, test_image_tensor)

        # Compute loss
        test_loss = F.mse_loss(test_pred[:batch_size].squeeze(), test_batch['building'].y[:batch_size].squeeze()).to('cpu')

        # Aggregate validation loss
        total_test_loss += test_loss.item() * batch_size
        
final_test_loss = total_test_loss / len(test_bids)
print(f'Test Loss: {round(final_test_loss, 5)}')


In [ ]:
all_loader = NeighborLoader(data, num_neighbors=[8,8], batch_size=batch_size, input_nodes = ('building'), shuffle=False, drop_last=False)

H, W = 512, 512
transforms = v2.Compose([
    v2.Resize([H,W]),
    v2.ToTensor()
])

all_bids = list(building_nodes['bid'].values)

class ImageDataset(Dataset):
    """Custom image dataset."""

    def __init__(self, root_dir, choice=None, transform=None):
        """
        Arguments:
            root_dir (string): Directory with all the images.
            choice (list): Accepts 'train', 'val', or 'test'
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.root_dir = root_dir
        self.transform = transform
        self.choice = choice
        self.targets = None
        
        if choice in ['train', 'test', 'val']:
            bids = np.loadtxt(os.path.join(os.getcwd(), 'energy_data', f'{self.choice}_bids'), dtype='str')
            self.paths = [os.path.join(os.getcwd(), self.root_dir, f'{bid}') for bid in bids]
            # self.targets = np.loadtxt(os.path.join(os.getcwd(), 'data', f'{self.choice}_targets'))
        if choice == 'pseudo_train':
            bids = np.loadtxt(os.path.join(os.getcwd(), 'energy_data', f'{self.choice}_bids'), dtype='str')
            self.paths = [os.path.join(os.getcwd(), self.root_dir, f'{bid}') for bid in bids]
        if choice == 'pseudo_train_val':
            bids = np.loadtxt(os.path.join(os.getcwd(), 'energy_data', f'{self.choice}_bids'), dtype='str')
            self.paths = [os.path.join(os.getcwd(), self.root_dir, f'{bid}') for bid in bids]
        if choice is None:
            bids =  all_bids
            self.paths = [os.path.join(os.getcwd(), self.root_dir, f'{bid}') for bid in bids]

    def __len__(self):
        return len(self.paths)
        
    def __getitem__(self, idx):
        if isinstance(idx,int):
            img_path = self.paths[idx] + '.png'
            image = Image.open(img_path)
            # target = self.targets[idx]
            if self.transform:
                image = self.transform(image)
                image = torch.unsqueeze(image, 0)
        
            return image
        
        if len(idx) > 1:
            temp = torch.Tensor()
            # target_list = []
            for ind in idx:
                img_path = self.paths[ind] + '.png'
                image = Image.open(img_path)
                # target = self.targets[ind]
                # target_list.append(target)
                if self.transform:
                    image = self.transform(image)
                    image = torch.unsqueeze(image, 0)
                temp = torch.concat((temp, image), dim=0)
                
            return temp
                
all_image_data = ImageDataset('./seattle_buildings/', transform = transforms)

In [ ]:
all_pred_targets = []
model.eval()
with torch.no_grad():
    for y,all_batch in enumerate(tqdm(all_loader)):
        # Prepare image tensor
        all_image_tensor = all_image_data[all_batch['building']['n_id'][0:all_batch['building']['batch_size']]].to(device)
        all_batch = all_batch.to(device)
        all_pred = model(all_batch.x_dict, all_batch.edge_index_dict, all_image_tensor)
        all_pred_targets+=all_pred[:batch_size].double().squeeze().tolist()

for y,all_batch in enumerate(tqdm(all_loader)):
    if y == (len(all_loader)-1):
        # Prepare image tensor
        all_image_tensor = all_image_data[all_batch['building']['n_id'][0:batch_size]].to(device)
        all_batch = all_batch.to(device)
        all_pred = model(all_batch.x_dict, all_batch.edge_index_dict, all_image_tensor)
        final_targets = all_pred[:batch_size].double().squeeze().tolist()[:all_batch['building']['batch_size']]

all_pred_targets += final_targets

In [41]:
np.savetxt('../energy_data/seattle_predictions', all_pred_targets)